In [1]:
import utils as u
import change as c
import ap50 as ap
import PySimpleGUI as sg
import detectron2
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import ColorMode, Visualizer
import glob
from detectron2 import model_zoo
from detectron2.data import DatasetCatalog, MetadataCatalog
import re
import cv2
from PIL import Image, ImageTk
import io
import os
import os.path
import time
import numpy as np


file_types = [("JPEG (*.jpg)", "*.jpg"),
              ("All files (*.*)", "*.*")]

#config_file = r"None"
#ckpt_file = r"Please read your check point file"
#img_file = r"Please read your img path"
#image_files = r"Please read your img path"

# 建立 detectron2 的 predictor
#cfg = model_zoo.get_config('COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml')
#cfg.merge_from_list(["MODEL.DEVICE", "cpu"])
#cfg.MODEL.WEIGHTS = "/Users/david/Desktop/DIP_final/outputModels1/model_final.pth"
#cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5 

#predictor = DefaultPredictor(cfg)

file_types = [("JPEG (*.jpg)", "*.jpg"),
              ("All files (*.*)", "*.*")]
file_list_column = [
    [
        sg.Listbox(
            values=[], enable_events=True, size=(40, 30), key="-FILE LIST-"
        )
    ],
]
height, width = 360, 360
def main():
    
    layout = [
        #sg.VSeperator(),
		#[sg.Text('config file',size=(16,1)), sg.In(config_file,size=(80,1), key='config'), sg.FileBrowse(file_types=file_types)],
        #[sg.Text('model checkpoint',size=(16,1), auto_size_text=False), sg.In(ckpt_file,size=(80,1), key='ckpt'), sg.FileBrowse(file_types=file_types)],
		#[sg.Text('Path to image',size=(16,1)), sg.In(img_file,size=(80,1), key='image'), sg.FileBrowse(file_types=file_types),sg.Button("Load Image"),],
        
        [
        sg.Text("Image Folder"),
        sg.In(size=(80, 1), enable_events=True, key="-FOLDER-"),
        sg.FolderBrowse('瀏覽'),
        ],
		[sg.Text('Device',size=(10,1)), sg.Combo(['cuda:0','cpu'],default_value='cpu',key='device')],
        [sg.Text(size=(40, 1), key="-TOUT-")],
		[sg.Text("File list",size=(10,1)),sg.Text("Source Image",size=(50,1)),sg.Text("Detected Image",size=(65,1)),sg.Text("Segmented Image",size=(65,1))],
        [sg.Column(file_list_column), sg.Image(key="source_image",size=(height,width),background_color='white'),
        sg.Image(key="detected_image",size=(height,width),background_color='white'),
        sg.Image(key="segmented_image",size=(height,width),background_color='white')],
        [ sg.Button('Detect'), sg.Button('Segment'),sg.Exit()],
        [sg.Text('IOU',size=(4,1)),  sg.In(size=(100,1), key='iou')],
        [sg.Text('AP50',size=(4,1)),  sg.In(size=(100,1), key='ap')],
        [sg.Text('Dice',size=(4,1)),  sg.In(size=(100,1), key='dice')],
        [sg.Text('FPS',size=(4,1)),  sg.In(size=(100,1), key='fps')]
        ]
    window = sg.Window('detectron2', 
                   layout,
                   default_element_size=(14,2),
                   text_justification='right',
                   auto_size_text=False
                   # element_justification='c'
                   )

    while True:
        event, values = window.read()

        if event == "Exit" or event == sg.WIN_CLOSED:
            break

        if event == "-FOLDER-":
            
            folder = values["-FOLDER-"]
            #print(folder.replace('image','label'))
            #c.get_data_dicts(folder.replace('image','label'))
            window["ap"].update(ap.get_ap50(folder.replace('image','label'))*100)#calculate ap50 takes a little long time
           
            try:
                file_list = os.listdir(folder)
            except:
                file_list = []

            fnames = [
                f
                for f in file_list
                if os.path.isfile(os.path.join(folder, f))
                and f.lower().endswith((".png", ".gif"))
            ]
            window["-FILE LIST-"].update(fnames)
        elif event == "-FILE LIST-":  
            try:
                filename = os.path.join(
                    values["-FOLDER-"], values["-FILE LIST-"][0]
                )
                #print(filename)
                image = Image.open(filename)
                image.thumbnail((400, 400))
                bio = io.BytesIO()
                # Actually store the image in memory in binary 
                image.save(bio, format="PNG")
                # Use that image data in order to 
                window["source_image"].update(data=bio.getvalue())
            except:
                pass


        if event == "Load Image":
            #start_time = time.time()  # 記錄目前的時間
            filename = values["image"]
            image_name = os.path.basename(filename)
            if os.path.exists(filename):
                image = Image.open(values["image"])
                image.thumbnail((400, 400))
                bio = io.BytesIO()
                # Actually store the image in memory in binary 
                image.save(bio, format="PNG")
                # Use that image data in order to 
                window["source_image"].update(data=bio.getvalue())

        if event == "image":
            file_path = values["image"]
            img = cv2.imread(file_path)
            # 將圖片轉換成 PySimpleGUI 可以顯示的格式
            img_bytes = cv2.imencode(".png", img)[1].tobytes()
            # 將圖片顯示在 PySimpleGUI 的視窗中
            window["source_image"].update(data=img_bytes)

            
        if event == "Segment":
            start_time = time.time()  # 記錄目前的時間
            filename = os.path.join(
                    values["-FOLDER-"], values["-FILE LIST-"][0]
                )
            # 使用 detectron2 偵測物件
            #filename = values["image"]
            label_path = filename.replace("image", "label")
            mask_path = filename.replace("image", "mask")
            cfg = get_cfg()
            
            # 載入模型
            microcontroller_metadata = { "name": "category_test", "thing_classes": ["powder_uncover", "powder_uneven", "scratch"]}
            cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
            cfg.MODEL.WEIGHTS = "/Users/david/Desktop/DIP_final/segmentation/outputModels/model_final.pth"
            cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
            cfg.MODEL.DEVICE = 'cpu'
            cfg.DATASETS.TEST = ("category_test", )
            cfg.MODEL.ROI_HEADS.NUM_CLASSES = 3
            #  創建預測器
            predictor = DefaultPredictor(cfg) 
            # 使用 detectron2 偵測圖片中的物體
            target_file = os.path.basename(mask_path)
            mask_dir = os.path.dirname(mask_path)#檢查相似檔案名
            checkfiles = os.listdir(mask_dir)
            pattern = re.compile(r'\d+')
            
            for file in checkfiles:
                result = pattern.search(file)
                if result and pattern.search(file).group() == pattern.search(target_file).group():
                    os.rename(os.path.join(mask_dir, file), os.path.join(mask_dir, target_file))#將最相近的檔案名改名為目標檔案名
                    mask_path = os.path.join(mask_dir, target_file)


            img = cv2.imread(mask_path)
            img = cv2.resize(img,(400,400))
            #iou_num = u.IOU_file(label_path[:-3] + 'json')
            outputs = predictor(img)
            
            v = Visualizer(img[:, :, ::-1],
                        metadata=microcontroller_metadata, 
                        scale=1, 
                        instance_mode=ColorMode.IMAGE_BW # removes the colors of unsegmented pixels
            )
            v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
            mask_array = outputs['instances'].to("cpu").pred_masks.numpy()
            num_instances = mask_array.shape[0]     
            mask_array = np.moveaxis(mask_array, 0, -1)  
            mask_array_instance = []
            output = np.zeros_like(img)   

            for i in range(num_instances):
                mask_array_instance.append(mask_array[:, :, i:(i+1)])
                output = np.where(mask_array_instance[i] == True, 255, output)
            # 將圖片轉換成 PySimpleGUI 可以顯示的格式
            #print(output)

            
            img = img.astype(np.float32)
            output = output.astype(np.float32)
            #print(dice_coef(img, output))
            img_bytes = cv2.imencode(".png", output)[1].tobytes()
            # 將圖片顯示在 PySimpleGUI 的視窗中
            #window["iou"].update(iou_num)
            elapsed_time = time.time() - start_time  # 計算執行時間
            window["dice"].update(u.dice_coef(img, output))
            #window["ap"].update(ap50(label_path))
            window["fps"].update(1/elapsed_time)
            window["segmented_image"].update(data=img_bytes)
        
        if event == "Detect":
            start_time = time.time()  # 記錄目前的時間
            filename = os.path.join(
                    values["-FOLDER-"], values["-FILE LIST-"][0]
                )
            # 使用 detectron2 偵測物件
            #filename = values["image"]
            label_path = filename.replace("image", "label")
            cfg = get_cfg()
            microcontroller_metadata = { "name": "category_test", "thing_classes": ["powder_uncover", "powder_uneven", "scratch"]}
            cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))
            cfg.MODEL.WEIGHTS = "/Users/david/Desktop/DIP_final/outputModels1/model_0029999.pth"
            cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
            cfg.MODEL.DEVICE = 'cpu'
            cfg.DATASETS.TEST = ("category_test", )
            cfg.MODEL.ROI_HEADS.NUM_CLASSES = 3
            predictor = DefaultPredictor(cfg) 
            # 使用 detectron2 偵測圖片中的物體
            img = cv2.imread(filename)
            img = cv2.resize(img,(400,400))
            iou_num = u.IOU_file(label_path[:-4] + '.json')
            outputs = predictor(img)
            v = Visualizer(img[:, :, ::-1],
                        metadata=microcontroller_metadata, 
                        scale=1, 
                        instance_mode=ColorMode.IMAGE_BW # removes the colors of unsegmented pixels
            )
            v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
            # 將圖片轉換成 PySimpleGUI 可以顯示的格式
    
            img_bytes = cv2.imencode(".png", v.get_image())[1].tobytes()
            # 將圖片顯示在 PySimpleGUI 的視窗中
            window["iou"].update(iou_num)
            
            #window["ap"].update(ap.get_ap50(os.path.dirname(label_path)))
            elapsed_time = time.time() - start_time  # 計算執行時間
            window["fps"].update(1/elapsed_time)
            window["detected_image"].update(data=img_bytes)


            
    window.close()


if __name__ == "__main__":

    main()


Mac OS Version is 13.0 and patch enabled so applying the patch
Applyting Mac OS 12.3+ Alpha Channel fix.  Your default Alpha Channel is now 0.99


2023-01-10 20:15:19.300 python[58153:5603527] +[CATransaction synchronize] called within transaction
2023-01-10 20:15:19.394 python[58153:5603527] +[CATransaction synchronize] called within transaction
/Users/david/opt/anaconda3/envs/detectron2/lib/python3.8/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ../c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


In [ ]:
import cv2
import numpy as np
from detectron2.config import get_cfg
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.utils.visualizer import Visualizer
from detectron2.data import DatasetCatalog, MetadataCatalog


In [ ]:
cfg = get_cfg()
# 載入模型
microcontroller_metadata = { "name": "category_test", "thing_classes": ["powder_uncover", "powder_uneven", "scratch"]}
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.WEIGHTS = "/Users/david/Desktop/DIP_final/segmentation/outputModels/model_final.pth"
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
cfg.MODEL.DEVICE = 'cpu'
cfg.DATASETS.TEST = ("category_test", )
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 3
#  創建預測器
predictor = DefaultPredictor(cfg)

In [ ]:
# 讀取圖像
import matplotlib.pyplot as plt
from detectron2.utils.visualizer import ColorMode
image = cv2.imread("/Users/david/Desktop/DIP_final/segmentation/data/train_mask/converted_ 0348.png")
outputs = predictor(image)
v = Visualizer(image[:, :, ::-1],
                   metadata=microcontroller_metadata, 
                   scale=1, 
                   instance_mode=ColorMode.IMAGE_BW # removes the colors of unsegmented pixels
)
v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    #v = v.get_output()
image = cv2.cvtColor(v.get_image()[:, :, ::-1], cv2.COLOR_BGR2RGB)

    #cv2.imwrite(out_path, image)
plt.figure(figsize = (14, 10))
plt.imshow(cv2.cvtColor(v.get_image()[:, :, ::-1], cv2.COLOR_BGR2RGB))